In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
conn=sqlite3.connect("database.sqlite")
c=conn.cursor()

In [3]:
#full description of the tables inside the database that we have
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,Tweets,Tweets,2,CREATE TABLE Tweets (\n tweet_id INTEGER PR...
1,table,normalized_tweets,normalized_tweets,4921,"CREATE TABLE normalized_tweets(tweet_id,airlin..."


In [22]:
pd.read_sql("SELECT * FROM Tweets;",conn)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567588278875213824,neutral,1.0000,,,Delta,,JetBlueNews,,0,@JetBlue's new CEO seeks the right balance to ...,,2015-02-16 23:36:05 -0800,USA,Sydney
1,567590027375702016,negative,1.0000,Can't Tell,0.6503,Delta,,nesi_1992,,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada)
2,567591480085463040,negative,1.0000,Late Flight,0.346,United,,CPoutloud,,0,@united yes. We waited in line for almost an h...,,2015-02-16 23:48:48 -0800,"Washington, DC",
3,567592368451248130,negative,1.0000,Late Flight,1,United,,brenduch,,0,@united the we got into the gate at IAH on tim...,,2015-02-16 23:52:20 -0800,,Buenos Aires
4,567594449874587648,negative,1.0000,Customer Service Issue,0.3451,Southwest,,VahidESQ,,0,@SouthwestAir its cool that my bags take a bit...,,2015-02-17 00:00:36 -0800,"Los Angeles, CA",Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,,,Delta,,Oneladyyouadore,,0,@JetBlue I hope so because I fly very often an...,,2015-02-24 11:48:29 -0800,Georgia,Quito
14481,570309340952993796,neutral,1.0000,,,US Airways,,DebbiMcGinnis,,0,@USAirways is a DM possible if you aren't foll...,,2015-02-24 11:48:37 -0800,Missourah,Hawaii
14482,570309345281486848,positive,0.6469,,,Delta,,jaxbra,,0,@JetBlue Yesterday on my way from EWR to FLL j...,,2015-02-24 11:48:38 -0800,"east brunswick, nj",Atlantic Time (Canada)
14483,570310144459972608,negative,1.0000,Customer Service Issue,1,US Airways,,GAKotsch,,0,@USAirways and when will one of these agents b...,,2015-02-24 11:51:48 -0800,,Atlantic Time (Canada)


In [4]:
pd.read_sql("""SELECT COUNT(airline_sentiment), airline_sentiment FROM Tweets 
                GROUP BY airline_sentiment;""",conn)

,COUNT(airline_sentiment),airline_sentiment
0,9082,negative
1,3069,neutral
2,2334,positive


In [21]:
pd.read_sql("""SELECT COUNT(airline_sentiment) FROM Tweets 
                WHERE airline_sentiment_confidence>0.7 AND airline_sentiment="positive" ;""",conn)

,COUNT(airline_sentiment)
0,1558


In [13]:
pd.read_sql("""SELECT airline, airline_sentiment, airline_sentiment_confidence FROM Tweets 
                WHERE airline_sentiment="neutral" ORDER BY airline, airline_sentiment_confidence;""",conn)

,airline,airline_sentiment,airline_sentiment_confidence
0,American,neutral,0.3367
1,American,neutral,0.3384
2,American,neutral,0.3421
3,American,neutral,0.3429
4,American,neutral,0.3455
...,...,...,...
3064,Virgin America,neutral,1.0000
3065,Virgin America,neutral,1.0000
3066,Virgin America,neutral,1.0000
3067,Virgin America,neutral,1.0000


In [17]:
pd.read_sql("""SELECT MIN(airline_sentiment_confidence) as "min_confidence", 
                AVG(airline_sentiment_confidence) AS "avg_confidence",
                MAX(airline_sentiment_confidence) AS "max_confidence"
                FROM Tweets;""",conn)

,min_confidence,avg_confidence,max_confidence
0,0.335,0.899958,1


In [30]:
pd.read_sql("""SELECT airline_sentiment_gold,negativereason_gold,
               tweet_coord FROM Tweets;""",conn)

,airline_sentiment_gold,negativereason_gold,tweet_coord
0,,,
1,,,
2,,,
3,,,
4,,,
...,...,...,...
14480,,,
14481,,,
14482,,,
14483,,,


In [21]:
#get the schema of the table
pd.read_sql("PRAGMA table_info('Tweets');",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,tweet_id,INTEGER,0,None,1
1,1,airline_sentiment,TEXT,0,None,0
2,2,airline_sentiment_confidence,NUMERIC,0,None,0
3,3,negativereason,TEXT,0,None,0
4,4,negativereason_confidence,NUMERIC,0,None,0
5,5,airline,TEXT,0,None,0
6,6,airline_sentiment_gold,TEXT,0,None,0
7,7,name,TEXT,0,None,0
8,8,negativereason_gold,TEXT,0,None,0
9,9,retweet_count,INTEGER,0,None,0


In [42]:
pd.read_sql("""SELECT DISTINCT airline_sentiment, airline_sentiment_gold,
                negativereason, negativereason_gold FROM normalized_tweets;""",conn)

,airline_sentiment,airline_sentiment_gold,negativereason,negativereason_gold
0,neutral,,,
1,negative,,Can't Tell,
2,negative,,Late Flight,
3,negative,,Customer Service Issue,
4,negative,,Bad Flight,
5,negative,,Flight Booking Problems,
6,positive,,,
7,negative,,Lost Luggage,
8,negative,,Cancelled Flight,
9,negative,,Flight Attendant Complaints,


In [ ]:
#adding data from one column of one table to one column of another table is possible
#by ALTER TABLE ADD COLUMN, and then UPDATE TABLE SET. In the second step in order to copy data
#in the same order from one table to the other table, a refrence column should be used.
#This column could be the forigen key of one of the table which is the primary key 
#of the other one. However, if we aim to make change in a single table, we do not have 
#such set of keys and if we have primary key,we should use that one as a refrence 
#in the update step. If there is no primary key, first we should create primary key for the tables
#(those we want to copy data from and to) and then use these pair of keys as reference to updata
#data in the added column.
#Now we are in the latest case where there is no key:
#we should create two tables (one as source table to copy data from and the other as destination
#table to add a new column to it and bring data from one column of source table to new added column
#of the destination column)

In [ ]:
#important: to prevent the error "'NoneType' object is not iterable", do not use
#pandas to create, drop, update,... a table. Use it only to get some results (read table, 
#get the schema, keys,...)

In [3]:
#drop table:
c.execute("DROP TABLE IF EXISTS tweets1;")

In [4]:
#create table:
c.execute("""CREATE TABLE IF NOT EXISTS tweets1(id INTEGER PRIMARY KEY, tweet_id INTEGER,
                airline_sentiment TEXT, airline_sentiment_confidence NUMERIC,
                negativereason TEXT, negativereason_confidence NUMERIC,airline TEXT,
                name TEXT, retweet_count INTEGER,text TEXT,tweet_coord TEXT, 
                tweet_created TEXT,tweet_location TEXT,user_timezone TEXT);""")

In [5]:
#bring all data that we want from old table to new table
c.execute("""insert into tweets1(tweet_id,
                airline_sentiment, airline_sentiment_confidence,
                negativereason, negativereason_confidence,airline,
                name, retweet_count,text,tweet_coord, 
                tweet_created,tweet_location,user_timezone)
                SELECT tweet_id, airline_sentiment, 
                airline_sentiment_confidence,
                negativereason, negativereason_confidence,airline,
                name, retweet_count,text,tweet_coord, 
                tweet_created,tweet_location,user_timezone 
                FROM Tweets;""")

In [96]:
pd.read_sql("SELECT * FROM tweets1;",conn)

,id,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,1,567588278875213824,neutral,1.0000,,,Delta,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,,2015-02-16 23:36:05 -0800,USA,Sydney
1,2,567590027375702016,negative,1.0000,Can't Tell,0.6503,Delta,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada)
2,3,567591480085463040,negative,1.0000,Late Flight,0.346,United,CPoutloud,0,@united yes. We waited in line for almost an h...,,2015-02-16 23:48:48 -0800,"Washington, DC",
3,4,567592368451248130,negative,1.0000,Late Flight,1,United,brenduch,0,@united the we got into the gate at IAH on tim...,,2015-02-16 23:52:20 -0800,,Buenos Aires
4,5,567594449874587648,negative,1.0000,Customer Service Issue,0.3451,Southwest,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,,2015-02-17 00:00:36 -0800,"Los Angeles, CA",Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14480,14481,570309308937842688,neutral,0.6869,,,Delta,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,,2015-02-24 11:48:29 -0800,Georgia,Quito
14481,14482,570309340952993796,neutral,1.0000,,,US Airways,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,,2015-02-24 11:48:37 -0800,Missourah,Hawaii
14482,14483,570309345281486848,positive,0.6469,,,Delta,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,,2015-02-24 11:48:38 -0800,"east brunswick, nj",Atlantic Time (Canada)
14483,14484,570310144459972608,negative,1.0000,Customer Service Issue,1,US Airways,GAKotsch,0,@USAirways and when will one of these agents b...,,2015-02-24 11:51:48 -0800,,Atlantic Time (Canada)


In [13]:
#drop table:
c.execute("DROP TABLE IF EXISTS normalized_tweets;")

In [14]:
#create table:
c.execute("""CREATE TABLE normalized_tweets(id INTEGER PRIMARY KEY, 
                tweet_id INTEGER,
                airline_sentiment TEXT, airline_sentiment_confidence NUMERIC,
                negativereason TEXT, negativereason_confidence NUMERIC,airline TEXT,
                airline_sentiment_gold TEXT,name TEXT,negativereason_gold TEXT,
                retweet_count INTEGER,text TEXT,tweet_coord TEXT, 
                tweet_created TEXT,tweet_location TEXT,user_timezone TEXT);""")

In [15]:
#bring all data that we want from old table to new table
c.execute("""insert into normalized_tweets(tweet_id,
                airline_sentiment, airline_sentiment_confidence,
                negativereason, negativereason_confidence,airline,
                name, retweet_count,text,tweet_coord, 
                tweet_created,tweet_location,user_timezone)
                SELECT tweet_id, airline_sentiment, 
                airline_sentiment_confidence,
                negativereason, negativereason_confidence,airline,
                name, retweet_count,text,tweet_coord, 
                tweet_created,tweet_location,user_timezone 
                FROM Tweets;""")

In [ ]:
#In order to make fundamental changes on the table, we Create a new table,
#bring all data from old table to this table, make fundamental changes on new table
#some changes like removing some columns, change of the column name and table name
#can not be performed on a table 
#and should be considered during create new table

In [16]:
pd.read_sql("SELECT * FROM normalized_tweets;",conn)

,id,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,1,567588278875213824,neutral,1.0000,,,Delta,None,JetBlueNews,None,0,@JetBlue's new CEO seeks the right balance to ...,,2015-02-16 23:36:05 -0800,USA,Sydney
1,2,567590027375702016,negative,1.0000,Can't Tell,0.6503,Delta,None,nesi_1992,None,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada)
2,3,567591480085463040,negative,1.0000,Late Flight,0.346,United,None,CPoutloud,None,0,@united yes. We waited in line for almost an h...,,2015-02-16 23:48:48 -0800,"Washington, DC",
3,4,567592368451248130,negative,1.0000,Late Flight,1,United,None,brenduch,None,0,@united the we got into the gate at IAH on tim...,,2015-02-16 23:52:20 -0800,,Buenos Aires
4,5,567594449874587648,negative,1.0000,Customer Service Issue,0.3451,Southwest,None,VahidESQ,None,0,@SouthwestAir its cool that my bags take a bit...,,2015-02-17 00:00:36 -0800,"Los Angeles, CA",Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14480,14481,570309308937842688,neutral,0.6869,,,Delta,None,Oneladyyouadore,None,0,@JetBlue I hope so because I fly very often an...,,2015-02-24 11:48:29 -0800,Georgia,Quito
14481,14482,570309340952993796,neutral,1.0000,,,US Airways,None,DebbiMcGinnis,None,0,@USAirways is a DM possible if you aren't foll...,,2015-02-24 11:48:37 -0800,Missourah,Hawaii
14482,14483,570309345281486848,positive,0.6469,,,Delta,None,jaxbra,None,0,@JetBlue Yesterday on my way from EWR to FLL j...,,2015-02-24 11:48:38 -0800,"east brunswick, nj",Atlantic Time (Canada)
14483,14484,570310144459972608,negative,1.0000,Customer Service Issue,1,US Airways,None,GAKotsch,None,0,@USAirways and when will one of these agents b...,,2015-02-24 11:51:48 -0800,,Atlantic Time (Canada)


In [17]:
#add new column to the table
c.execute("ALTER TABLE normalized_tweets ADD COLUMN new_col;")

In [18]:
c.execute("""UPDATE normalized_tweets 
            SET (new_col) = (SELECT tweets1.text FROM tweets1
            WHERE normalized_tweets.id = tweets1.id);""")

In [99]:
#check the new table
pd.read_sql("SELECT * FROM normalized_tweets;",conn)

,id,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,new_col
0,1,567588278875213824,neutral,1.0000,,,Delta,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,,2015-02-16 23:36:05 -0800,USA,Sydney,@JetBlue's new CEO seeks the right balance to ...
1,2,567590027375702016,negative,1.0000,Can't Tell,0.6503,Delta,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada),@JetBlue is REALLY getting on my nerves !! 😡😡 ...
2,3,567591480085463040,negative,1.0000,Late Flight,0.346,United,CPoutloud,0,@united yes. We waited in line for almost an h...,,2015-02-16 23:48:48 -0800,"Washington, DC",,@united yes. We waited in line for almost an h...
3,4,567592368451248130,negative,1.0000,Late Flight,1,United,brenduch,0,@united the we got into the gate at IAH on tim...,,2015-02-16 23:52:20 -0800,,Buenos Aires,@united the we got into the gate at IAH on tim...
4,5,567594449874587648,negative,1.0000,Customer Service Issue,0.3451,Southwest,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,,2015-02-17 00:00:36 -0800,"Los Angeles, CA",Pacific Time (US & Canada),@SouthwestAir its cool that my bags take a bit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14480,14481,570309308937842688,neutral,0.6869,,,Delta,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,,2015-02-24 11:48:29 -0800,Georgia,Quito,@JetBlue I hope so because I fly very often an...
14481,14482,570309340952993796,neutral,1.0000,,,US Airways,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,,2015-02-24 11:48:37 -0800,Missourah,Hawaii,@USAirways is a DM possible if you aren't foll...
14482,14483,570309345281486848,positive,0.6469,,,Delta,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,,2015-02-24 11:48:38 -0800,"east brunswick, nj",Atlantic Time (Canada),@JetBlue Yesterday on my way from EWR to FLL j...
14483,14484,570310144459972608,negative,1.0000,Customer Service Issue,1,US Airways,GAKotsch,0,@USAirways and when will one of these agents b...,,2015-02-24 11:51:48 -0800,,Atlantic Time (Canada),@USAirways and when will one of these agents b...


In [67]:
#important: using sqlite3 library in python, both these queries remove the whole records
#thus rename the table is only possible by create new table with the new name,copy data
#from old table to new table
#important: the same statement is valid for renaming and removing the column

#change name of the table
#c.execute("ALTER normalized_tweets RENAME TO tweets_less;")

#change name of the column
#c.execute("ALTER normalized_tweets RENAME tweet_created TO tweet_time;")

#remove a column
#c.execute("ALTER TABLE normalized_tweets DROP COLUMN new_col")

In [20]:
#update the values of some columns:
c.execute("""UPDATE normalized_tweets 
                SET airline_sentiment="positive" 
                WHERE airline_sentiment_confidence=1;""")

In [92]:
pd.read_sql("SELECT airline_sentiment_confidence,airline_sentiment FROM normalized_tweets;",conn)

,airline_sentiment_confidence,airline_sentiment
0,1.0000,positive
1,1.0000,positive
2,1.0000,positive
3,1.0000,positive
4,1.0000,positive
...,...,...
14480,0.6869,neutral
14481,1.0000,positive
14482,0.6469,positive
14483,1.0000,positive


In [21]:
#create view:
c.execute("""CREATE TEMP VIEW IF NOT EXISTS view_normalized_tweets
            (airline_sentiment_confidence, negativereason_confidence,airline, name)
            AS SELECT airline_sentiment_confidence, negativereason_confidence,airline, name
            FROM normalized_tweets;""")

In [89]:
pd.read_sql("SELECT * FROM view_normalized_tweets;",conn)

,airline_sentiment_confidence,negativereason_confidence,airline,name
0,1.0000,,Delta,JetBlueNews
1,1.0000,0.6503,Delta,nesi_1992
2,1.0000,0.346,United,CPoutloud
3,1.0000,1,United,brenduch
4,1.0000,0.3451,Southwest,VahidESQ
...,...,...,...,...
14480,0.6869,,Delta,Oneladyyouadore
14481,1.0000,,US Airways,DebbiMcGinnis
14482,0.6469,,Delta,jaxbra
14483,1.0000,1,US Airways,GAKotsch


In [22]:
#remove an index:
c.execute("""DROP INDEX IF EXISTS index_normalized_tweets;""")

In [23]:
#create index of the table:
c.execute("""CREATE INDEX index_normalized_tweets 
                ON normalized_tweets(airline_sentiment_confidence,
                negativereason_confidence,airline, name);""")

In [24]:
#get the list of indices associated with a specific table
pd.read_sql("""PRAGMA index_list('normalized_tweets');""",conn)

,seq,name,unique,origin,partial
0,0,index_normalized_tweets,0,c,0


In [25]:
#get the details about a specific index(method1)
pd.read_sql("""PRAGMA index_info('index_normalized_tweets');""",conn)

,seqno,cid,name
0,0,3,airline_sentiment_confidence
1,1,5,negativereason_confidence
2,2,6,airline
3,3,8,name


In [26]:
#get the details about a specific index(method2)
pd.read_sql("""SELECT type, name, tbl_name, sql FROM sqlite_master WHERE type= 'index';""",conn)

,type,name,tbl_name,sql
0,index,index_normalized_tweets,normalized_tweets,CREATE INDEX index_normalized_tweets \n ...
